## Setup: Imports, Retry Config

In [ ]:
from pathlib import Path
import sys
import sqlite3

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext

print("✅ ADK components imported successfully.")

Add the project root to sys.path to easily import our classes:

In [ ]:
# Ensure the project root (the parent of the "src" directory) is on sys.path
# so that "import src.model" finds the src package under the project root.
project_root = Path.cwd().parent
src_dir = project_root / "src"

project_root_path = str(project_root.resolve())
if project_root_path not in sys.path:
    sys.path.insert(0, project_root_path)

from src.model import Intensity, SentimentOutput
from src.model.rag_output import RagOutput
from src.config import load_env_variables, get_env_variable

In [ ]:
load_env_variables()

GOOGLE_API_KEY = get_env_variable("GOOGLE_API_KEY")
print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

OPENROUTER_API_KEY = get_env_variable("OPENROUTER_API_KEY")
print("OPENROUTER_API_KEY loaded:", bool(OPENROUTER_API_KEY))

Create a general retry policy:

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [ ]:
# RAG system
rag_agent = Agent(
    name="RAG_Vaccine_Informer",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="rag_output",  # The result of this agent will be stored in the session state with this key.
    output_schema=RagOutput,  # Define the expected output schema
)

print("✅ RAG created.")

## Engine Def

In [ ]:
APP_NAME = "VaccineInfoRAG"
SQL_ASYNC_DRIVER = "aiosqlite"
DB_NAME = "vaxtalk_sessions.db"
DB_URL = f"sqlite+{SQL_ASYNC_DRIVER}:///{DB_NAME}"  # Local SQLite file

Session Management:

In [ ]:
session_service = InMemorySessionService()

Application:

In [ ]:
application = App(
    name=APP_NAME,
    root_agent=rag_agent
)

In [ ]:
runner = Runner(
    app=application, 
    session_service=session_service
)

## Testing

In [ ]:
response = await runner.run_debug(
    "Should I vaccinate?"
)

## Exploring Persisted State

Let's explore the state we saved on the database.
Notice that there are special **Compaction Events**, created with our compaction policy to summarize the context.

In [ ]:
def check_data_in_db():
    with sqlite3.connect(DB_NAME) as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

If we knew the session ID, we could explore the events directly from the session service:

In [ ]:
user_id = "user"
session_id = "debug_session_id"

# Get the final session state
final_session = await session_service.get_session(
    app_name=application.name,
    user_id=user_id,
    session_id=session_id,
)
if not final_session:
    raise ValueError("Final session not found.")

print("Searching for Compaction Summary Event")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("✅ SUCCESS! Found the Compaction Event:")
        print(f"Author: {event.author}")
        print(f"Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print("❌ No compaction event found.")